In [1]:
import os
import openai

openai.api_key = "YOUR_API_KEY"
# or
# openai.api_key = os.environ["OPENAI_API_KEY"]

def llm(prompt):
    payload=  [
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=payload
    )
    return response['choices'][0]['message']['content']

llm("What color is the sky?")

"The color of the sky can vary depending on various factors such as time of day, weather conditions, and location. During the day, the sky appears blue because of the way Earth's atmosphere scatters sunlight. However, at sunrise or sunset, the sky can appear orange, pink, red, or purple due to the scattering of shorter wavelength light. At night, the sky appears predominantly black, but can also have shades of blue or other colors when light pollution or atmospheric conditions are present."

In [2]:
import yaml

def load_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return yaml.dump(data)

load_yaml('environment.yaml')

'environment:\n  house:\n    bedroom:\n      bed:\n        description: A queen-sized bed with a thick duvet and several pillows.\n      description: A cozy room with dim lighting and plush carpeting.\n      wardrobe:\n        description: A tall wooden wardrobe with sliding doors and a mirror on one\n          side.\n    description: A two-story suburban home with a white picket fence.\n    kitchen:\n      cabinet:\n        description: Oak wood cabinets, some of which are high up near the ceiling.\n      description: A modern kitchen with stainless steel appliances.\n      fridge:\n        description: A double-door stainless steel fridge with a water dispenser.\n    living_room:\n      couch:\n        description: A large beige sectional sofa, big enough to hide behind.\n      description: A spacious room with a large window looking out to the garden.\n      tv_stand:\n        description: A modern black TV stand with several compartments below.\n'

In [3]:
def generate_persona(manual=False, persona=None):
    return llm("Generate a human persona that will be used in a hide and seek game. You will play as the seeker, searching for the hiders. Your response should be a few sentences long. Ensure it is in the second person.")

In [4]:
import re

def validate_and_format_strategy(prompt_path, memories_path, regex_pattern, environment_string, quick_probability, cost_map, max_cost, persona):

    with open(prompt_path, 'r') as file:
        seeker_prompt = file.read()

    with open(memories_path, 'r') as file:
        seeker_memories = file.read()
    
    # Insert the memories and environment data into the seeker_prompt
    replacements = {
    '!<MEMORY>!': seeker_memories,
    '!<ENVIRONMENT>!': environment_string,
    '!<QUICK_PROBABILITY>!': str(quick_probability),
    '!<QUICK_COST>!': str(cost_map['quick']),
    '!<THOROUGH_COST>!': str(cost_map['thorough']),
    '!<TOTAL_POINTS>!': str(max_cost),
    '!<PERSONA>!': persona,
    }
    formatted_prompt = seeker_prompt
    for placeholder, replacement in replacements.items():
        formatted_prompt = formatted_prompt.replace(placeholder, replacement)

    output = llm(formatted_prompt)

    ct = 0
    while True:
    # Regex pattern to extract strategy
        matches = re.findall(regex_pattern, output)

        if matches:
            # Convert matches to a dictionary
            strategy_with_costs = [(location, thoroughness, cost_map[thoroughness]) for location, thoroughness in matches]
            
            filtered_strategy = {}
            total_cost = 0
            for location, thoroughness, cost in strategy_with_costs:
                if total_cost + cost <= max_cost:
                    filtered_strategy[location] = thoroughness
                    total_cost += cost
                else:
                    break
            
            return filtered_strategy
        else:
            print(f'Invalid strategy attempt: {ct}, retrying...')
            ct += 1
            output = llm(formatted_prompt)

# Example
prompt_path = 'seeker_prompt.txt'
memories_path = 'seeker_memories.txt'
cost_map = {
    'quick': 1,
    'thorough': 2
}
quick_probability = 0.5
regex_pattern = r'(house\..+?):\s(quick|thorough)'
environment_string = load_yaml('environment.yaml')
max_cost = 5

validate_and_format_strategy(prompt_path=prompt_path, memories_path=memories_path,
                              regex_pattern=regex_pattern, environment_string=environment_string, quick_probability=quick_probability,
                              cost_map=cost_map, max_cost=max_cost, persona="You use aggressive tactics to find the hider quickly.")

{'house.bedroom.wardrobe': 'thorough',
 'house.kitchen.cabinet': 'quick',
 'house.bedroom.bed': 'thorough'}

In [5]:
import random

def get_hider_location(hider_prompt_path, hider_memories_path, environment_string):

    with open(hider_prompt_path, 'r') as file:
        hider_prompt = file.read()

    with open(hider_memories_path, 'r') as file:
        hider_memories = file.read()

    formatted_prompt = hider_prompt.replace('!<MEMORY>!', hider_memories).replace('!<ENVIRONMENT>!', environment_string)
    hider_location = llm(formatted_prompt)
    # Test if the location is valid
    match = re.search(r'(house\.[a-zA-Z0-9_]+\.[a-zA-Z0-9_]+)', hider_location)
    ct = 1
    while not match:
        print(f"Invalid location, Attempt {ct}, Trying again...")
        hider_location = llm(formatted_prompt)
        match = re.search(r'(house\.[a-zA-Z0-9_]+\.[a-zA-Z0-9_]+)', hider_location)
        ct += 1
    hider_location = match.group(0)
    return hider_location

# Execute the strategy and determine success
def execute_strategy(strategy, quick_probability, hider_location):
    
    for location, thoroughness in strategy.items():
        if location == hider_location and thoroughness == "thorough":
            return "found"
        elif location == hider_location and thoroughness == "quick":
            if random.random() < quick_probability:
                return "found"
            else:
                return "not_found"

            
    return "not_found"

def get_next_iteration_number(memories_path):
    with open(memories_path, 'r') as f:
        lines = f.readlines()
    iteration_count = sum(1 for line in lines if line.startswith("iteration_"))
    return iteration_count + 1

def append_to_seeker_memory(seeker_memories_path, hider_location, strategy, outcome):
    iteration_number = get_next_iteration_number(seeker_memories_path)
    with open(seeker_memories_path, 'a') as f:
        f.write(f"iteration_{iteration_number}:\n")
        f.write(f"hider_location: {hider_location}\n")
        f.write(f"seeker_strategy: {strategy}\n")
        f.write(f"outcome: {outcome}\n\n")

def append_to_hider_memory(hider_memories_path, hider_location, outcome):
    iteration_number = get_next_iteration_number(hider_memories_path)
    with open(hider_memories_path, 'a') as f:
        f.write(f"iteration_{iteration_number}:\n")
        f.write(f"hider_location: {hider_location}\n")
        f.write(f"outcome: {outcome}\n\n")

def clear_memory(file_path):
    with open(file_path, 'w') as f:
        f.write('')

In [6]:
def run(n, seeker_memories_path='seeker_memories.txt', seeker_prompt_path='seeker_prompt.txt', hider_memories_path='hider_memories.txt', hider_prompt_path='hider_prompt.txt',
         quick_probability = 0.5,cost=4):
    results = []
    persona = generate_persona()
    print(f"Persona: {persona}")
    for i in range(n):
        hider_location = get_hider_location(hider_prompt_path, hider_memories_path, environment_string)
        strategy = validate_and_format_strategy(prompt_path=prompt_path, memories_path=memories_path,
                              regex_pattern=regex_pattern, environment_string=environment_string, quick_probability=quick_probability,
                              cost_map=cost_map, max_cost=max_cost, persona=persona)
        outcome = execute_strategy(strategy, quick_probability, hider_location)
        append_to_seeker_memory(seeker_memories_path, hider_location, strategy, outcome)
        append_to_hider_memory(hider_memories_path, hider_location, outcome)
        results.append((outcome, hider_location, strategy))
        print(f"Iteration {i+1}: {outcome}")
    next_step = input("Clear memories? (y/n)")
    if next_step == 'y':
        clear_memory(seeker_memories_path)
        clear_memory(hider_memories_path)
    else:
        print("Not clearing memories")
    return results

In [7]:
import numpy as np
import bqplot.pyplot as plt
from bqplot import Tooltip

def viz_results(games):
    # Calculate seeker's cumulative win percentage
    wins = np.cumsum([1 if game[0] == 'found' else 0 for game in games])
    total_games = np.arange(1, len(games) + 1)
    win_percentage = wins / total_games * 100  # Convert to percentage

    # Define tooltips
    def get_strategy(i):
        return ', '.join([f"{loc}: {strategy}" for loc, strategy in games[i][2].items()])

    # Visualization using bqplot
    fig_margin = dict(top=50, bottom=60, left=80, right=20)
    plt.figure(title='Seeker Cumulative Win Percentage', fig_margin=fig_margin)

    # Line plot for the win percentage
    plt.plot(total_games, win_percentage, colors=['blue'])

    # Tooltips for scatter plot
    tooltips = [
        f"Iteration: {i+1}\n" +
        f"Win Percentage: {win_percentage[i]:.2f}%\n" +
        f"Strategy: {get_strategy(i)}\n" +
        f"Hider Location: {games[i][1]}"
        for i in range(len(games))
    ]

    # Scatter plot for tooltips
    scatters = plt.scatter(total_games, win_percentage, colors=['blue'], default_size=40, names=tooltips, display_names=False)

    # Assign tooltips to scatter plot
    scatters.tooltip = Tooltip(fields=['name'], show_labels=False)

    plt.show()

In [8]:
games = run(8)

Persona: You are Agent Smith, a seasoned detective known for your sharp instincts and relentless pursuit. Dressed in a tailored suit, your steely eyes scan the surroundings as you search for the crafty hiders. With a reputation for always finding your targets, you rely on your keen observation skills and the ability to predict their next move. No corner, no shadow is safe from your watchful gaze, as you navigate the game of hide and seek with unwavering determination.
Iteration 1: found
Iteration 2: not_found
Iteration 3: not_found
Iteration 4: found
Iteration 5: not_found
Iteration 6: not_found
Iteration 7: not_found
Iteration 8: found


In [9]:
viz_results(games)